# House Construction Expense Tracker

This notebook helps you track and analyze expenses for your house construction project. You can:
1. Record expenses with categories and subcategories
2. View total expenses by category
3. Track spending patterns over time
4. Visualize expense distribution
5. Auto-save data to CSV and Excel
6. Create automatic backups

All data is stored securely in the `house_expenses_data` folder.

## 1. Import Required Libraries

First, let's import the necessary Python libraries for data management and visualization.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import os
import shutil

# Set the style for better visualizations
plt.style.use('seaborn')
sns.set_palette("husl")

## 2. Setup Expense Categories and Data Storage

Define the expense categories and setup data storage structure.

In [ ]:
# Define the data directory structure
DATA_DIR = 'house_expenses_data'
CSV_FILE = os.path.join(DATA_DIR, 'expenses.csv')
EXCEL_FILE = os.path.join(DATA_DIR, 'house_expenses.xlsx')
BACKUP_DIR = os.path.join(DATA_DIR, 'backups')

# Create necessary directories
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(BACKUP_DIR, exist_ok=True)

# Define expense categories and subcategories
expense_categories = {
    'Foundation': ['Excavation', 'Concrete', 'Waterproofing', 'Other'],
    'Structure': ['Framing', 'Roof', 'Windows', 'Doors', 'Other'],
    'Systems': ['Electrical', 'Plumbing', 'HVAC', 'Solar', 'Other'],
    'Interior': ['Flooring', 'Walls', 'Ceiling', 'Paint', 'Lighting', 'Other'],
    'Exterior': ['Siding', 'Landscaping', 'Driveway', 'Fencing', 'Other'],
    'Kitchen': ['Cabinets', 'Countertops', 'Appliances', 'Plumbing Fixtures', 'Other'],
    'Bathroom': ['Fixtures', 'Tiles', 'Vanities', 'Shower/Tub', 'Other'],
    'Permits': ['Building Permit', 'Inspection Fees', 'Other'],
    'Labor': ['Contractor Fees', 'Specialized Labor', 'General Labor', 'Other'],
    'Other': ['Materials', 'Tools', 'Rentals', 'Miscellaneous']
}

# Create an empty DataFrame to store expenses
expenses_df = pd.DataFrame(columns=[
    'Date',
    'Category',
    'Subcategory',
    'Description',
    'Amount',
    'Payment_Method',
    'Receipt_Reference'
])

# Print available categories and subcategories
print("Available Expense Categories:")
for category, subcategories in expense_categories.items():
    print(f"\n{category}:")
    for sub in subcategories:
        print(f"  - {sub}")

## 3. Data Management Functions

Create functions to:
- Add new expenses
- Save data to CSV and Excel
- Load existing data
- Create automatic backups

In [ ]:
def save_data():
    """
    Save the expense data to both CSV and Excel files with automatic backup
    """
    global expenses_df
    
    # Save to CSV
    expenses_df.to_csv(CSV_FILE, index=False)
    
    # Save to Excel with formatting
    with pd.ExcelWriter(EXCEL_FILE, engine='openpyxl') as writer:
        # Save main data
        expenses_df.to_excel(writer, sheet_name='Expenses', index=False)
        
        # Create summary sheet
        summary = pd.DataFrame({
            'Category': expenses_df.groupby('Category')['Amount'].sum().index,
            'Total Amount': expenses_df.groupby('Category')['Amount'].sum().values
        })
        summary.to_excel(writer, sheet_name='Summary', index=False)
        
        # Create monthly summary sheet
        monthly = expenses_df.set_index('Date').resample('M')['Amount'].sum().reset_index()
        monthly.to_excel(writer, sheet_name='Monthly Trends', index=False)
    
    # Create backup
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_file = os.path.join(BACKUP_DIR, f'expenses_backup_{timestamp}.xlsx')
    shutil.copy2(EXCEL_FILE, backup_file)
    
    print(f"Data saved successfully!")
    print(f"CSV file: {CSV_FILE}")
    print(f"Excel file: {EXCEL_FILE}")
    print(f"Backup created: {backup_file}")

def load_data():
    """
    Load expense data from CSV file
    """
    global expenses_df
    
    if os.path.exists(CSV_FILE):
        expenses_df = pd.read_csv(CSV_FILE)
        # Convert date strings back to datetime
        expenses_df['Date'] = pd.to_datetime(expenses_df['Date'])
        print("Data loaded successfully!")
        return True
    else:
        print("No existing data file found. Starting with empty dataset.")
        return False

def add_expense(date, category, subcategory, description, amount, payment_method="Cash", receipt_reference=""):
    """
    Add a new expense and automatically save the data
    """
    global expenses_df
    
    # Validate category and subcategory
    if category not in expense_categories:
        print(f"Error: Invalid category. Please choose from: {list(expense_categories.keys())}")
        return
    
    if subcategory not in expense_categories[category]:
        print(f"Error: Invalid subcategory for {category}. Please choose from: {expense_categories[category]}")
        return
    
    # Create a new row
    new_expense = pd.DataFrame({
        'Date': [pd.to_datetime(date)],
        'Category': [category],
        'Subcategory': [subcategory],
        'Description': [description],
        'Amount': [float(amount)],
        'Payment_Method': [payment_method],
        'Receipt_Reference': [receipt_reference]
    })
    
    # Add the new expense to the DataFrame
    expenses_df = pd.concat([expenses_df, new_expense], ignore_index=True)
    
    # Save the updated data
    save_data()
    print("Expense added and saved successfully!")

# Try to load existing data
load_data()

## 4. Expense Analysis and Visualization

Functions to analyze expenses and create visualizations for better insights.

In [ ]:
def analyze_expenses():
    """
    Analyze expenses and display summary statistics
    """
    if len(expenses_df) == 0:
        print("No expenses recorded yet!")
        return
    
    # Calculate total expenses
    total_expenses = expenses_df['Amount'].sum()
    
    # Calculate expenses by category
    category_expenses = expenses_df.groupby('Category')['Amount'].sum().sort_values(ascending=False)
    
    # Calculate expenses by subcategory
    subcategory_expenses = expenses_df.groupby(['Category', 'Subcategory'])['Amount'].sum()
    
    print(f"\nTotal Expenses: ${total_expenses:,.2f}")
    print("\nExpenses by Category:")
    print(category_expenses)
    print("\nDetailed Expenses by Subcategory:")
    print(subcategory_expenses)
    
    return {
        'total': total_expenses,
        'by_category': category_expenses,
        'by_subcategory': subcategory_expenses
    }

def visualize_expenses():
    """
    Create visualizations for expense distribution
    """
    if len(expenses_df) == 0:
        print("No expenses recorded yet!")
        return
    
    # Create a pie chart for category distribution
    fig1 = px.pie(expenses_df, values='Amount', names='Category', 
                  title='Expense Distribution by Category')
    fig1.show()
    
    # Create a bar chart for subcategory distribution
    category_sub = expenses_df.groupby(['Category', 'Subcategory'])['Amount'].sum().reset_index()
    fig2 = px.bar(category_sub, x='Subcategory', y='Amount', color='Category',
                  title='Expenses by Subcategory',
                  labels={'Amount': 'Total Amount ($)', 'Subcategory': 'Subcategory'})
    fig2.show()
    
    # Create a time series of cumulative expenses
    expenses_df['Cumulative_Total'] = expenses_df.sort_values('Date')['Amount'].cumsum()
    fig3 = px.line(expenses_df.sort_values('Date'), x='Date', y='Cumulative_Total',
                   title='Cumulative Expenses Over Time',
                   labels={'Cumulative_Total': 'Cumulative Total ($)', 'Date': 'Date'})
    fig3.show()

def generate_monthly_report():
    """
    Generate a monthly expense report
    """
    if len(expenses_df) == 0:
        print("No expenses recorded yet!")
        return
    
    # Monthly totals
    monthly_totals = expenses_df.set_index('Date').resample('M')['Amount'].sum()
    
    # Monthly category breakdown
    monthly_by_category = expenses_df.set_index('Date').pivot_table(
        values='Amount',
        columns='Category',
        aggfunc='sum',
        resample='M'
    ).fillna(0)
    
    print("\nMonthly Expense Totals:")
    print(monthly_totals)
    print("\nMonthly Breakdown by Category:")
    print(monthly_by_category)
    
    # Visualize monthly trends
    fig = px.bar(monthly_totals.reset_index(), x='Date', y='Amount',
                 title='Monthly Expense Totals',
                 labels={'Amount': 'Total Amount ($)', 'Date': 'Month'})
    fig.show()

## 5. Usage Examples

Here are some examples of how to use the expense tracker:

In [ ]:
# Example 1: Adding a new expense
add_expense(
    date="2025-06-01",
    category="Foundation",
    subcategory="Concrete",
    description="Initial foundation concrete work",
    amount=15000,
    payment_method="Bank Transfer",
    receipt_reference="FOUND001"
)

# Example 2: Adding another expense
add_expense(
    date="2025-06-01",
    category="Permits",
    subcategory="Building Permit",
    description="Building permit application fee",
    amount=2500,
    payment_method="Credit Card",
    receipt_reference="PERM001"
)

# Analyze expenses
analyze_expenses()

# Create visualizations
visualize_expenses()

# Generate monthly report
generate_monthly_report()

## 2. Define Expense Categories

Let's set up the categories and subcategories for tracking house construction expenses.